# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [23]:
import os
import glob
import numpy as np
import psycopg2
import json
from sql_queries import *
import pandas as pd


In [24]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [25]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [26]:
song_files = ""

In [27]:
filepath = "data/song_data"

In [28]:
song_files = get_files(filepath)

songs = []

for song_file in song_files:
    with open(song_file) as json_data:
        song_data = json.load(json_data)
        json_data.close()
        songs.append(song_data)

song_df = pd.DataFrame(songs)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [29]:
song_vals = song_df.get(["song_id", "title", "artist_id", "year", "duration"]).values

song_list = []

for song_val in song_vals:
    song_dict = {"song_id": song_val[0], "title": song_val[1], "artist_id": song_val[2], "year": song_val[3], "duration": song_val[4]}
    song_list.append(song_dict)
    
song_data = pd.DataFrame(song_list)
song_data.head()

,artist_id,duration,song_id,title,year
0,ARD7TVE1187B99BFB1,218.93179,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARMJAGH1187FB546F3,148.03546,SOCIWDW12A8C13D406,Soul Deep,1969
2,ARKRRTF1187B9984DA,177.47546,SOXVLOJ12AB0189215,Amor De Cabaret,0
3,AR7G5I41187FB4CE6C,233.40363,SONHOTT12A8C13493C,Something Girls,1982
4,ARXR32B1187FB57099,209.60608,SOFSOCN12A8C143F5D,Face the Ashes,2007


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [30]:
for song in song_data.itertuples():
    song_info_list = [song.song_id, song.title, song.artist_id, song.year, song.duration]
    cur.execute(song_table_insert, song_info_list)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [31]:
artist_vals = song_df.get(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).values

artist_list = []

for artist_val in artist_vals:
    artist_dict = {"artist_id": artist_val[0], "artist_name": artist_val[1], "artist_location": artist_val[2], "artist_latitude": artist_val[3], "artist_longitude": artist_val[4]}
    artist_list.append(artist_dict)
    
artist_data = pd.DataFrame(artist_list)
artist_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual
1,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops
2,ARKRRTF1187B9984DA,NaN,,NaN,Sonora Santanera
3,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant
4,ARXR32B1187FB57099,NaN,,NaN,Gob


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [32]:
for artist in artist_data.itertuples():
    artist_info_list = [artist.artist_id, artist.artist_name, artist.artist_location, artist.artist_latitude, artist.artist_longitude]
    cur.execute(artist_table_insert, artist_info_list)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [33]:
log_files = ""

In [34]:
filepath = "data/log_data"

In [35]:
log_files = get_files(filepath)

logs = []

count = 0
for log_file in log_files:
    with open(log_file) as fp:
        line = fp.readline()
        line = line.strip()
        if len(line) > 0:
            json_log = json.loads(line)
            logs.append(json_log)
        while line:
            line = fp.readline()
            line = line.strip()
            if len(line) > 0:
                json_log = json.loads(line)
                logs.append(json_log)
                
df = pd.DataFrame(logs)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` 
to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [36]:
filtered_df = df.loc[df['page'].isin(["NextSong"])]
ts_vals = filtered_df.get(["ts"]).values

In [37]:
time_list = []

for ts_val in ts_vals:
    dt = pd.to_datetime(ts_val[0], unit='ms')
    dt_dict = {"start_time": dt, "hour": dt.hour, "day": dt.day, "week": dt.weekofyear, "month": dt.month, "year": dt.year, "weekday": dt.weekday()}
    time_list.append(dt_dict)    

In [38]:
time_df = pd.DataFrame(time_list)

In [39]:
time_df.head(1)

,day,hour,month,start_time,week,weekday,year
0,1,21,11,2018-11-01 21:01:46.796,44,3,2018


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [40]:
for ts_data in time_df.itertuples():
    time_list = [ts_data.start_time, ts_data.hour, ts_data.day, ts_data.week, ts_data.month, ts_data.year, ts_data.weekday]
    cur.execute(time_table_insert, time_list)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [41]:
user_vals = df.get(["userId", "firstName", "lastName", "gender", "level"]).values

user_list = []

for user_val in user_vals:
    user_dict = {"user_id": user_val[0], "first_name": user_val[1], "last_name": user_val[2], "gender": user_val[3], "level": user_val[4]}
    user_list.append(user_dict)
    
user_df = pd.DataFrame(user_list)
user_df.head()

,first_name,gender,last_name,level,user_id
0,Walter,M,Frye,free,39
1,Kaylee,F,Summers,free,8
2,Kaylee,F,Summers,free,8
3,Kaylee,F,Summers,free,8
4,Kaylee,F,Summers,free,8


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [42]:
for user_data in user_df.itertuples():
    user_list = [user_data.user_id, user_data.first_name, user_data.last_name, user_data.gender, user_data.level]
    cur.execute(user_table_insert, user_list)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [43]:
for index, row in df.iterrows():
    cur.execute(song_select, [row.song, row.artist, row.length])
    results = cur.fetchall()
    if len(results) > 0:
        songid, artistid = results[0]
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.